In [ ]:
import pandas as pd
import numpy as np 
import keras 
from keras.layers import Dense, Dropout

C:\Users\Szymon\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Szymon\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Szymon\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Szymon\Anaconda3\lib\site-packages\tensorflow\python\framew

In [2]:
from keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D

In [3]:
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

In [35]:
from statistics import mean
import matplotlib.pyplot as plt

In [5]:
from sklearn.model_selection import train_test_split

In [87]:
headers = ['sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'decision']
data = pd.read_csv('output_cumulated.csv', names=headers)
data.head()

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,decision
0,398.0,101.857902,60.925663,474.156302,231.819254,straight
1,396.0,101.542193,60.925663,472.084688,231.819254,straight
2,394.0,101.542193,60.925663,470.015662,231.819254,straight
3,392.0,101.542193,60.925663,467.944048,231.819254,straight
4,390.0,101.542193,60.925663,465.875022,231.819254,straight


In [88]:
df = pd.get_dummies(data, columns=['decision'])

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15003 entries, 0 to 15002
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sensor_1           15003 non-null  float64
 1   sensor_2           15003 non-null  float64
 2   sensor_3           15003 non-null  float64
 3   sensor_4           15003 non-null  float64
 4   sensor_5           15003 non-null  float64
 5   decision_left      15003 non-null  uint8  
 6   decision_right     15003 non-null  uint8  
 7   decision_straight  15003 non-null  uint8  
dtypes: float64(5), uint8(3)
memory usage: 630.1 KB


In [90]:
x = df[list(df.columns)[:5]]
y = df[list(df.columns)[5:]]

In [91]:
X_train, X_test, y_train, y_test = train_test_split(x, y, shuffle=True, test_size=0.33, random_state=42, stratify=data['decision'])

In [92]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

X_train = np.clip(X_train, -5, 5)
X_test = np.clip(X_test, -5, 5)

In [93]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [114]:
# class_weights={0: 1 - sum([el[0] for el in y_train]) / len(y_train), # left
#                1: 1 - sum([el[1] for el in y_train]) / len(y_train), # right
#                2: 1 - sum([el[2] for el in y_train]) / len(y_train), # straight
#               }
class_weights={0: 10, # left
               1: 10, # right
               2: 1, # straight
              }

In [115]:
class_weights

{0: 10, 1: 10, 2: 1}

# Model creation


In [107]:
from keras import optimizers

In [144]:
def build_model():
    network = keras.Sequential()
    network.add(Dense(2000, activation='relu', input_shape=(5,)))
    network.add(Dropout(0.15))
    network.add(Dense(2000, activation='relu', input_shape=(5,)))
    network.add(Dropout(0.15))
    network.add(Dense(3, activation='softmax'))
    optimizer = optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    network.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    return network

In [109]:
from keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

def build_model_2():
    K.clear_session()
    inputs = Input(shape=(5,1))
    #First Conv1D layer
    conv = Conv1D(45,3, padding='valid', activation='relu', strides=1)(inputs)
    conv = MaxPooling1D(3)(conv)
    conv = Dropout(0.3)(conv)
    
    #Flatten layer
    conv = Flatten()(conv)

    #Dense Layer 1
    conv = Dense(4500, activation='relu')(conv)
    conv = Dropout(0.3)(conv)
    outputs = Dense(3, activation='softmax')(conv)

    model = Model(inputs, outputs)
    model.compile(loss='categorical_crossentropy',optimizer='rmsprob',metrics=['accuracy'])
    return model

In [99]:
def plot_loss_curve(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(loss) + 1)

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()
    

In [100]:
def evaluate_classifier(model_f, train_data, train_targets, epochs, class_weights):
    k = 5
    num_val_samples = len(train_data) // k
    num_epochs = epochs
    all_scores = []
    for i in range(k):
        print('processing fold #', i)
        # Prepare the validation data: data from partition # k
        val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
        val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

        # Prepare the training data: data from all other partitions
        partial_train_data = np.concatenate(
            [train_data[:i * num_val_samples],
             train_data[(i + 1) * num_val_samples:]],
            axis=0)
        partial_train_targets = np.concatenate(
            [train_targets[:i * num_val_samples],
             train_targets[(i + 1) * num_val_samples:]],
            axis=0)

        # Build the Keras model (already compiled)
        model = model_f()
        # Train the model (in silent mode, verbose=0)
        history = model.fit(partial_train_data, partial_train_targets,
                  epochs=num_epochs, batch_size=32, verbose=1, class_weight=class_weights, validation_data=(val_data, val_targets))
        # Evaluate the model on the validation data
        val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1)
        all_scores.append(val_mae)
        pred = model.predict(val_data)
        
        print(f"Mean absolute error: {val_mae}\n")
        
        plt.figure(figsize=(16,8))
        plt.plot(history.history['loss'], 'bo')
        plt.plot(history.history['val_loss'], 'b')
        plt.title('Whole training and validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        #plt.ylim((0, 15))
        plt.legend()

        plt.show()
    return mean(all_scores)

In [73]:
evaluate_classifier(build_model, X_train, y_train, 50, class_weights)

processing fold # 0
Train on 11794 samples, validate on 2948 samples
Epoch 1/50
11794/11794 [==============================] - 1s 125us/step - loss: 0.6286 - accuracy: 0.7622 - val_loss: 0.6130 - val_accuracy: 0.7619
Epoch 2/50
11794/11794 [==============================] - 1s 83us/step - loss: 0.5894 - accuracy: 0.7705 - val_loss: 0.6074 - val_accuracy: 0.7659
Epoch 3/50
11794/11794 [==============================] - 1s 83us/step - loss: 0.5805 - accuracy: 0.7712 - val_loss: 0.5945 - val_accuracy: 0.7636
Epoch 4/50
11794/11794 [==============================] - 1s 83us/step - loss: 0.5706 - accuracy: 0.7758 - val_loss: 0.5768 - val_accuracy: 0.7710
Epoch 5/50
11794/11794 [==============================] - 1s 83us/step - loss: 0.5664 - accuracy: 0.7757 - val_loss: 0.5704 - val_accuracy: 0.7727
Epoch 6/50
11794/11794 [==============================] - 1s 83us/step - loss: 0.5629 - accuracy: 0.7752 - val_loss: 0.5889 - val_accuracy: 0.7697
Epoch 7/50
11794/11794 [========================

KeyboardInterrupt: 

In [152]:
model = build_model()

In [165]:
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test))

Train on 10052 samples, validate on 4951 samples
Epoch 1/200
10052/10052 [==============================] - 3s 266us/step - loss: 0.3533 - accuracy: 0.8547 - val_loss: 0.4681 - val_accuracy: 0.8352
Epoch 2/200
10052/10052 [==============================] - 2s 237us/step - loss: 0.3493 - accuracy: 0.8526 - val_loss: 0.4601 - val_accuracy: 0.8406
Epoch 3/200
10052/10052 [==============================] - 2s 235us/step - loss: 0.3514 - accuracy: 0.8522 - val_loss: 0.4626 - val_accuracy: 0.8352
Epoch 4/200
10052/10052 [==============================] - 2s 235us/step - loss: 0.3496 - accuracy: 0.8522 - val_loss: 0.4596 - val_accuracy: 0.8330
Epoch 5/200
10052/10052 [==============================] - 2s 235us/step - loss: 0.3491 - accuracy: 0.8534 - val_loss: 0.4701 - val_accuracy: 0.8338
Epoch 6/200
10052/10052 [==============================] - 2s 246us/step - loss: 0.3503 - accuracy: 0.8559 - val_loss: 0.4713 - val_accuracy: 0.8402
Epoch 7/200
10052/10052 [==============================] 

KeyboardInterrupt: 

In [163]:
predictions = model.predict(X_test)

In [164]:
pred_labels = [np.argmax(el) for el in predictions]
true_labels = [np.argmax(el) for el in y_test]
cm = confusion_matrix(true_labels, pred_labels)
print("Confusion matrix:\n{}".\
      format(cm))

Confusion matrix:
[[ 382   10  386]
 [   6  304  258]
 [  80   56 3469]]


In [150]:
model.save('trained_model_test.h5')

In [230]:
from pickle import dump

In [231]:
dump(scaler, open('scaler.pkl', 'wb'))